In [1]:
from utility_bfsolver import *

In [3]:
counts = pd.read_pickle("/Users/zihengchen/Documents/Analysis/workplace/data/count/count_.pkl")

In [3]:
slt = []
for i in range(4):
    slv = BFSolver3D(counts.acc[i])
    obsX = slv.setMeasuredX(counts.ndata[i],counts.nmcbg[i]+counts.nfake[i])
    slt.append(slv.solveQuadEqn(obsX))
slt = np.array(slt)
slt

array([[0.10906   , 0.11190817, 0.11046231],
       [0.11311639, 0.11377278, 0.10780647],
       [0.10557132, 0.10544763, 0.10428385],
       [0.10835108, 0.10847748, 0.10495039]])

In [43]:
# solver 1D; error from BWe and BWm
errm = []
erre = []
for i in range(4):
    slv = BFSolver1D(counts.acc[i])
    slve = BFSolver1D(counts.acc[i],bWe=0.1071-0.0016, bWm=0.1063)
    slvm = BFSolver1D(counts.acc[i],bWe=0.1071, bWm=0.1063-0.0015)
    obsX = slv.setMeasuredX(counts.ndata[i],counts.nmcbg[i]+counts.nfake[i])
    
    errm.append(slvm.solveQuadEqn(obsX) - slv.solveQuadEqn(obsX))
    erre.append(slve.solveQuadEqn(obsX) - slv.solveQuadEqn(obsX))
errm = np.array(errm).reshape(-1)[np.array([1,2,4,5,6,8,9,11])]/0.1086 * 100
erre = np.array(erre).reshape(-1)[np.array([1,2,4,5,6,8,9,11])]/0.1086 * 100


In [58]:
# solver 1D: error from e,mu,tau eff
err = []
effUp = np.array([0,0,0.05,0]) + 1
for i in range(4):
    slv = BFSolver1D(counts.acc[i])
    slv1 = BFSolver1D(counts.acc[i] * effUp[:,None,None])
    
    BW = slv .solveQuadEqn(slv .setMeasuredX(counts.ndata[i],counts.nmcbg[i]+counts.nfake[i]))
    BW1= slv1.solveQuadEqn(slv1.setMeasuredX(counts.ndata[i],counts.nmcbg[i]* effUp+counts.nfake[i]))


    
    err.append(BW1-BW)
err = np.array(err).reshape(-1)[np.array([1,2,4,5,6,8,9,11])]/0.1086 * 100

In [60]:
print(list(abs(err)))

[0.4196690492852734, 0.1044862536329217, 0.4002783550551415, 0.0869299614802864, 7.7861199516056185, 0.0913757511240937, 7.482788365788479, 0.0732900164289858]


In [12]:
err = []        
for i in range(4):
    slv = BFSolver1D(counts.acc[i])
    BW  = slv.solveQuadEqn(slv.setMeasuredX(counts.ndata[i],counts.nmcbg[i]+counts.nfake[i]))

    # taking derivertive dBW/dnData
    deltaNData = np.identity(4)
    dBW_over_dnData = []
    for iDeltaNData in range(4):
        BW1 = slv.solveQuadEqn(slv.setMeasuredX( counts.ndata[i]+deltaNData[iDeltaNData],
                                                 counts.nmcbg[i]+counts.nfake[i]))
        dBW_over_dnData.append(BW1-BW)
    dBW_over_dnData = np.array(dBW_over_dnData)
    # propagating error
    errFromSource = np.matmul(counts.ndataVar[i], dBW_over_dnData**2)**0.5
    err.append(errFromSource)
err = np.array(err).reshape(-1)[np.array([1,2,4,5,6,8,9,11])]/0.1086 * 100

In [14]:
print(list(abs(err)))

[2.4917381068282967, 1.2158241216449708, 4.320565446921318, 2.0172081169243605, 4.861123962410149, 1.4037731328891698, 8.573592506996826, 2.4006219993556894]
